In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def relu(x, derivative = False):
    res = x
    if derivative == True:
        return (res > 0)*1
    else:
        return res * (res > 0)

def sigmoid(x, derivative = False):
    res = 1/(1 + np.exp(-x))
    if derivative == True:
        return res * (1 - res)
    else:
        return res
    
def first_moment_update(previous_moment, grad, beta, timestep):
    biased = beta * previous_moment + (1 - beta) * grad
    unbiased = biased / (1 - np.power(beta, timestep))
    return unbiased

def second_moment_update(previous_moment, grad, beta, timestep):
    biased = beta * previous_moment + (1 - beta) * np.square(grad)
    unbiased = biased / (1 - np.power(beta,timestep))
    return unbiased

def params_to_vector(dictionary):
    counter = 0
    for i in dictionary:
        if counter == 0:
            new_vec = dictionary[i].reshape(-1)
        else:
            new_vec2 = dictionary[i].reshape(-1)
            new_vec = np.concatenate([new_vec, new_vec2])
        counter += 1
    return new_vec

def forward_propagation(inputs, parameters):
    W_1, b_1, W_mu, b_mu, W_sigma, b_sigma, W_2, b_2 = parameters.values()
    m = inputs.shape[1]
    
    z_1 = W_1.dot(inputs) + b_1
    a_1 = relu(z_1, derivative = False)
    
    z_mu = W_mu.dot(a_1) + b_mu
    z_sigma = W_sigma.dot(a_1) + b_sigma

    eps = np.random.randn(n_z, len(inputs.T))
    sampled_vector = z_mu + np.multiply(np.exp(z_sigma * 0.5), eps) 

    z_2 = W_2.dot(sampled_vector) + b_2
    
    cost = np.mean(np.square(inputs - z_2))
    
    cache = (W_1, b_1, W_mu, b_mu, W_sigma, b_sigma, W_2, b_2, z_1, a_1, z_mu, z_sigma, sampled_vector, z_2)
    
    return cost, cache

def backward_propagation(inputs, cache):
    W_1, b_1, W_mu, b_mu, W_sigma, b_sigma, W_2, b_2, z_1, a_1, z_mu, z_sigma, sampled_vector, z_2 = cache
    
    m = inputs.shape[1]
    grad_z_2 = 2* (z_2 - inputs) # (n_x, m)
    grad_W_2 = (1/m) * grad_z_2.dot(sampled_vector.T) # (n_z, m)
    grad_b_2 = (1/m) * np.sum(grad_z_2, axis=1, keepdims = True)
    grad_s =  W_2.T.dot(grad_z_2) # (n_z, m), gradient of sampled vector
    grad_z_mu = grad_s # (n_z, m)
    grad_z_sigma = np.multiply(grad_s, 0.5 * np.exp(z_sigma * 0.5) * eps)
    grad_W_mu = (1/m) * grad_z_mu.dot(a_1.T)
    grad_b_mu = (1/m) * np.mean(grad_z_mu, axis = 1,keepdims=True)
    grad_W_sigma = grad_z_sigma.dot(a_1.T)
    grad_b_sigma = np.mean(grad_z_sigma, axis = 1, keepdims=True)
    grad_a_1 = W_mu.T.dot(grad_z_mu)
    grad_z_1 = np.multiply(grad_a_1, relu(z_1, derivative=True))
    grad_b_1 = (1/m) * np.mean(grad_z_1, axis = 1, keepdims = True)
    grad_W_1 = (1/m) * grad_z_1.dot(batch.T)
    
    grads['W_1'] = grad_W_1
    grads['b_1'] = grad_b_1
    grads['W_mu'] = grad_W_mu
    grads['b_mu'] = grad_b_mu
    grads['W_sigma'] = grad_W_sigma
    grads['b_sigma'] = grad_b_sigma
    grads['W_2'] = grad_W_2
    grads['b_2'] = grad_b_2
    
    return grads

In [3]:
n_x = 1 # Number of features
m = 128 # Number of data points
l_1 = 20 # Number of neurons in 1st layer
n_z = 10 # Number of latent variables
alpha = 1e-1 # learning_rate
epochs = 1
batch_size = 128
kl_weight = 5e-3

# Adam parameters
beta_1 = 0.9
beta_2 = 0.999
little_eps = 1e-8

In [4]:
# Testing that everything works before converting to a class
np.random.seed(0)
train_inputs = np.random.randn(m, n_x)
train_inputs = train_inputs.T

learned_params = {}
# learned_params['W_1'] = np.random.randn(l_1, n_x) * np.sqrt(2/n_x) # Kaiming initialization
learned_params['W_1'] = np.random.uniform(low = - np.sqrt(6 / (l_1 + n_x)),
                                          high = np.sqrt(6 / (l_1 + n_x)),
                                          size = (l_1, n_x)) # Xavier
learned_params['b_1'] = np.zeros(shape = (l_1,1))
# learned_params['W_mu'] = np.random.randn(n_z, l_1) * np.sqrt(2/l_1) # Kaiming initialization
learned_params['W_mu'] = np.random.uniform(low = - np.sqrt(6 / (l_1 + n_z)),
                                           high = np.sqrt(6 / (l_1 + n_z)),
                                           size = (n_z, l_1)) # Xavier
learned_params['b_mu'] = np.zeros(shape = (n_z, 1))
# learned_params['W_sigma'] = np.random.randn(n_z, l_1) * np.sqrt(2/l_1) # Kaiming initialization
learned_params['W_sigma'] = np.random.uniform(low = -np.sqrt(6 / (l_1 + n_z)),
                                              high = np.sqrt(6 / (l_1 + n_z)),
                                              size = (n_z, l_1)) # Xavier
learned_params['b_sigma'] = np.zeros(shape = (n_z,1))
# learned_params['W_2'] = np.random.randn(n_x, n_z) * np.sqrt(2/n_z) # Kaiming initialization
learned_params['W_2'] = np.random.uniform(low = -np.sqrt(6 / (n_x + n_z)),
                                          high = np.sqrt(6 / (n_x + n_z)),
                                          size = (n_x, n_z)) # Xavier
learned_params['b_2'] = np.zeros(shape = (n_x, 1))

# Still seems too big, so I'm going to dock everything by 1e-2
# learned_params = {i:j*1e-1 for i, j in learned_params.items()}

learnable_params = learned_params.keys()

first_moments = {i:0 for i in learnable_params}
second_moments = {i:0 for i in learnable_params}
grads = {i:0 for i in learnable_params}

# train_truth = (np.random.sample(size=(n_x,1)) >= 0.5)*1

losses_mse = []
losses_kl = []
losses = []

# Implement mini-batch gradient descent.
batches_per_epoch = (m // batch_size) + 1 - (1 * (isinstance(m / batch_size, int)))
timestep = 1 # Keeps track of how many updates have been done, used by Adam optimizer.

for i in range(epochs):
    for j in range(1):
        if j == (batches_per_epoch - 1): # Last batch
            batch = train_inputs[:, j* batch_size:]
        else:
            batch = train_inputs[:, j * batch_size: (j+1) * batch_size] # All features, current batch
        
        W_1, b_1, W_mu, b_mu, W_sigma, b_sigma, W_2, b_2 = learned_params.values() # Get the current set of values.
            
        # Forward pass
        z_1 = W_1.dot(batch) + b_1
        a_1 = relu(z_1)

        z_mu = W_mu.dot(a_1) + b_mu
        # There is no activation for mu layer
        z_sigma = W_sigma.dot(a_1) + b_sigma
        
        eps = np.random.randn(n_z, len(batch.T)) # Get the number of rows
        sampled_vector = z_mu + np.multiply(np.exp(z_sigma * 0.5), eps) # Elem-wise multiplication for eps and var.
        # Also treat z_sigma as a log-var instead, bypass the problem with negatives.

        z_2 = W_2.dot(sampled_vector) + b_2
        # Should there be a final activation? 
        # I don't think so - if we're going for MSE loss we're going to compare every single feature vector with the 
        # reconstructed one. So the range of the activation function has to be the whole real line, but most common ones
        # are not that way, like ReLU, tanh, sigmoid.

        # Loss - go with simple L2 loss
        loss_mse = np.mean(np.square(batch - z_2)) # One number
        loss_kl = 0.5*(np.sum(np.exp(z_sigma) + np.square(z_mu) - 1 - z_sigma, axis=0)) # m numbers
        loss = loss_mse + kl_weight * loss_kl # (m,) shape

        losses_mse.append(loss_mse)
        losses_kl.append(np.mean(loss_kl))
        losses.append(loss_mse + np.mean(loss_kl))
        # Warning - KL loss, taking mean is obscuring how some are close to N(0,1) and some are not.

#         if i % 5 == 0:
#             print('The loss from epoch {} batch {} is: '.format(i,j) + str(loss))

        # Backward pass
        # Need one for W_1, b_1, W_mu, b_mu, W_sigma, b_sigma, W_2, b_2
        # This is for MSE loss      
        grad_z_2 = 2* (z_2 - batch) # (n_x, m)
        grad_W_2 = grad_z_2.dot(sampled_vector.T) # (n_z, m)
        grad_b_2 = np.mean(grad_z_2, axis=1, keepdims = True)
        grad_s =  W_2.T.dot(grad_z_2) # (n_z, m), gradient of sampled vector
        grad_z_mu = grad_s # (n_z, m)
        grad_z_sigma = np.multiply(grad_s, 0.5 * np.exp(z_sigma * 0.5) * eps)
        grad_W_mu = grad_z_mu.dot(a_1.T)
        grad_b_mu = np.mean(grad_z_mu, axis = 1,keepdims=True)
        grad_W_sigma = grad_z_sigma.dot(a_1.T)
        grad_b_sigma = np.mean(grad_z_sigma, axis = 1, keepdims=True)
        grad_a_1 = W_mu.T.dot(grad_z_mu)
        grad_z_1 = np.multiply(grad_a_1, relu(z_1, derivative=True))
        grad_b_1 = np.mean(grad_z_1, axis = 1, keepdims = True)
        grad_W_1 = grad_z_1.dot(batch.T)

#         # This is for KL loss
#         grad_z_sigma +=  np.exp(z_sigma) - 1
#         grad_z_mu += 2*z_mu # Seriously, wth. There is likely something from KL that can give intuition on this.
#         grad_W_sigma += grad_z_sigma.dot(a_1.T)
#         grad_b_sigma += np.squeeze(np.sum(grad_z_sigma, keepdims = True))
#         grad_W_mu += grad_z_mu.dot(a_1.T)
#         grad_b_mu += np.squeeze(np.sum(grad_z_mu, keepdims = True))

        # Store all the gradients (Anyway to write this code cleaner?)
        grads['W_1'] = grad_W_1
        grads['b_1'] = grad_b_1
        grads['W_mu'] = grad_W_mu
        grads['b_mu'] = grad_b_mu
        grads['W_sigma'] = grad_W_sigma
        grads['b_sigma'] = grad_b_sigma
        grads['W_2'] = grad_W_2
        grads['b_2'] = grad_b_2
        
#         for param in learnable_params:
#             learned_params[param] -= alpha * grads[param]
        alpha *= np.sqrt(1-np.power(beta_2,timestep)) / (1 - np.power(beta_1,timestep))
        for param in learnable_params:
            
            first_moments[param] = first_moment_update(previous_moment = first_moments[param], 
                                                       grad = grads[param],
                                                       beta = beta_1,
                                                       timestep = timestep)
            second_moments[param] = second_moment_update(previous_moment = second_moments[param],
                                                         grad = grads[param],
                                                         beta = beta_2,
                                                         timestep = timestep)
            
            learned_params[param] -= alpha * np.divide(first_moments[param],(np.sqrt(second_moments[param]) + little_eps))
    timestep += 1

In [23]:
epsilon = 1e-7
parameters_values = params_to_vector(learned_params)
grad = params_to_vector(grads)
num_parameters = len(parameters_values)
J_plus = np.zeros((num_parameters,1))
J_minus = np.zeros((num_parameters,1))
gradapprox = np.zeros((num_parameters,1))

for i in range(num_parameters):
    thetaplus = np.copy(parameters_values)
    thetaplus[i] = thetaplus[i] + epsilon
    J_plus[i], _ = forward_propagation(train_inputs, learned_params)
    
    thetaminus = np.copy(parameters_values)
    thetaminus[i] = thetaminus[i] - epsilon
    J_minus[i], _ = forward_propagation(train_inputs, learned_params)
    
    gradapprox[i] = (J_plus[i] - J_minus[i]) / (2 *(epsilon))
    
# Compare gradapprox to backward propagation gradients
numerator = np.linalg.norm(grad - gradapprox)
denominator = np.linalg.norm(grad) + np.linalg.norm(gradapprox)
numerator/denominator

21.701392988438776

### Matching TF and base
### Forward propagation

In [4]:
import keras
import tensorflow as tf

tf.enable_eager_execution() # Must be called at program startup?

from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Lambda
from tensorflow import set_random_seed

# Somehow if you use keras.layers it'll throw an error. I guess if you are using eager execution, use tensorflow
# everywhere. Side: Stuff this.

import numpy as np

import matplotlib.pyplot as plt

def sampling(args):
    z_mean, z_logsigma = args
    tf.set_random_seed(0)
    epsilon = tf.random_normal(shape = tf.shape(z_mean))
    sampled_vector = tf.add(z_mean, tf.multiply(tf.exp(.5 * z_logsigma), epsilon))
    return sampled_vector

def total_vae_loss (x, x_pred, mu, logsigma, kl_weight =5e-3):
    kl_loss = 0.5 * tf.reduce_sum(tf.exp(logsigma) + tf.square(mu) - 1 - logsigma, axis = 1)
    reconstruction_loss = tf.reduce_mean((x - x_pred)**2)
    total_vae_loss = kl_weight * kl_loss + reconstruction_loss
    
    losses = {'kl_loss': kl_loss,
              'rc_loss': reconstruction_loss,
              'total_vae_loss': total_vae_loss}
    return losses

set_random_seed(0)
inputs = Input(shape = n_x, batch_size = batch_size) # Have to put batch size first, eh.

a_1 = Dense(units = l_1, activation = 'relu')(inputs)
z_mean = Dense(units = n_z)(a_1)
z_logsigma = Dense(units = n_z)(a_1)

# Implement the sampling layer
sampled_vector = Lambda(sampling)([z_mean, z_logsigma])

z_2 = Dense(units = n_x)(sampled_vector)

model = Model(inputs = inputs, outputs = [z_2, z_mean, z_logsigma])

print(model.summary())

# model.compile(optimizer = tf.train.AdamOptimizer(learning_rate = alpha),
#               loss = total_vae_loss) # No idea how to make loss work yet

# np.random.seed(0)
# train_inputs = np.random.randn(m, n_x)
# train_inputs = tf.convert_to_tensor(train_inputs, dtype = np.float32)
# optimizer = tf.train.AdamOptimizer(learning_rate = alpha)
# total_loss_values = []
# rc_loss_values = []
# kl_loss_values = []

# batches_per_epoch = (m // batch_size) + 1 - (1 * (isinstance(m / batch_size, int)))

# for i in range(1):
#     for j in range(1):
#         if j == (batches_per_epoch - 1): # Last batch
#             batch = train_inputs[j* batch_size:, :]
#         else:
#             batch = train_inputs[j * batch_size: (j+1) * batch_size, :] # All features, current batch
            
#         with tf.GradientTape() as tape:
#             z_2, z_mean, z_logsigma = model(batch)
#             losses = total_vae_loss(batch, z_2, z_mean, z_logsigma)
#             grads = tape.gradient(losses['total_vae_loss'], model.weights) # model.weights means variables here.
#             optimizer.apply_gradients(zip(grads, model.weights), global_step = tf.train.get_or_create_global_step())

#             # Track the losses
#             total_loss_values.append(losses['total_vae_loss'].numpy().mean()) # loss_value is a tensor with loss for every single data point.
#             rc_loss_values.append(losses['rc_loss'].numpy().mean())
#             kl_loss_values.append(losses['kl_loss'].numpy().mean())

Using TensorFlow backend.


Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(128, 1)]           0                                            
__________________________________________________________________________________________________
dense (Dense)                   (128, 20)            40          input_1[0][0]                    
__________________________________________________________________________________________________
dense_1 (Dense)                 (128, 10)            210         dense[0][0]                      
__________________________________________________________________________________________________
dense_2 (Dense)                 (128, 10)            210         dense[0][0]                      
______________________________________________________________________________________________

In [5]:
np.random.seed(0)
train_inputs = np.random.randn(m, n_x)

# To be the same as tensorflow we have no choice but to initialize using its model weights, and steal it.
learned_params = {}
learned_params['W_1'] = model.get_weights()[0]
learned_params['b_1'] = model.get_weights()[1]
learned_params['W_mu'] = model.get_weights()[2]
learned_params['b_mu'] = model.get_weights()[3]
learned_params['W_sigma'] = model.get_weights()[4]
learned_params['b_sigma'] = model.get_weights()[5]
learned_params['W_2'] = model.get_weights()[6]
learned_params['b_2'] = model.get_weights()[7]

# for i in range(len(learned_params)):
#     assert(learned_params[list(learned_params.keys())[i]].shape == model.get_weights()[i].shape)

Next step - put both through forward propagation.

Note that in Keras, the forward propagation is done in an X.W manner - see this:
https://github.com/keras-team/keras/blob/master/keras/layers/core.py. In particular, we have the call method to Dense layer.

Thus, we change our feedforward code to transpose the learned parameters.

In [6]:
def forward_propagation_K(inputs, parameters, kl_weight = 5e-3):
    W_1, b_1, W_mu, b_mu, W_sigma, b_sigma, W_2, b_2 = parameters.values()
    
    z_1 = np.dot(inputs, W_1) + b_1
    a_1 = relu(z_1)
    z_mu = np.dot(a_1, W_mu) + b_mu
    z_sigma = np.dot(a_1, W_sigma) + b_sigma
    
    eps = tf.random_normal(shape = tf.shape(z_mu)).numpy()
    sampled_vector = z_mu + np.multiply(np.exp(z_sigma * 0.5), eps)
    
    z_2 = np.dot(sampled_vector, W_2) + b_2
    
    cost = np.mean(np.square(inputs - z_2))
    loss_kl = 0.5*(np.sum(np.exp(z_sigma) + np.square(z_mu) - 1 - z_sigma, axis=1))
    total_vae_loss = cost + kl_weight * loss_kl
    cache = {
        'W_1': W_1,
        'b_1': b_1,
        'z_1': z_1,
        'a_1': a_1,
        'W_mu': W_mu,
        'b_mu': b_mu,
        'z_mu': z_mu,
        'W_sigma': W_sigma,
        'b_sigma': b_sigma,
        'z_sigma': z_sigma,
        'sampled_vector': sampled_vector,
        'W_2': W_2,
        'b_2': b_2,
        'z_2': z_2,
        'eps': eps,
        'inputs':inputs,
    }
    return total_vae_loss, cache

tf.set_random_seed(0)
forward_propagation_K(train_inputs, learned_params)[0]

array([1.65901931, 1.65351872, 1.65497699, 1.662714  , 1.65973859,
       1.65525709, 1.6548752 , 1.65327835, 1.65325238, 1.65353405,
       1.6532671 , 1.65713101, 1.65428151, 1.6532564 , 1.65358553,
       1.65343048, 1.6573518 , 1.653319  , 1.65340641, 1.65477762,
       1.66720818, 1.654004  , 1.65458949, 1.65439815, 1.66296949,
       1.65772897, 1.65323356, 1.65330406, 1.65757258, 1.65721393,
       1.65327296, 1.65348774, 1.6549023 , 1.6616034 , 1.65348636,
       1.65327375, 1.65600629, 1.65587999, 1.6535478 , 1.6534235 ,
       1.65556418, 1.65751819, 1.65943154, 1.66034969, 1.65378048,
       1.65363662, 1.65656496, 1.65432782, 1.6587751 , 1.65332572,
       1.65493141, 1.65349984, 1.65378298, 1.65619086, 1.65323148,
       1.65356099, 1.65323774, 1.65339463, 1.65408306, 1.65350871,
       1.65418883, 1.65350382, 1.65463259, 1.65957868, 1.65328641,
       1.65357199, 1.65888823, 1.65361662, 1.65497674, 1.65323464,
       1.65419447, 1.6532597 , 1.65560615, 1.6564697 , 1.65352

In [7]:
def total_vae_loss (x, x_pred, mu, logsigma, kl_weight =5e-3):
    kl_loss = 0.5 * tf.reduce_sum(tf.exp(logsigma) + tf.square(mu) - 1 - logsigma, axis = 1)
    reconstruction_loss = tf.reduce_mean((x - x_pred)**2)
    total_vae_loss = kl_weight * kl_loss + reconstruction_loss
    
    losses = {'kl_loss': kl_loss,
              'rc_loss': reconstruction_loss,
              'total_vae_loss': total_vae_loss}
    return losses
tf.set_random_seed(0)
z_2, z_mu, z_sigma = model(train_inputs)
losses = total_vae_loss(train_inputs, z_2, z_mu, z_sigma)
losses['total_vae_loss']

<tf.Tensor: id=318, shape=(128,), dtype=float32, numpy=
array([1.6590192, 1.6535187, 1.654977 , 1.6627139, 1.6597385, 1.655257 ,
       1.6548752, 1.6532782, 1.6532522, 1.6535339, 1.653267 , 1.657131 ,
       1.6542814, 1.6532563, 1.6535854, 1.6534303, 1.6573517, 1.6533189,
       1.6534063, 1.6547775, 1.6672081, 1.6540039, 1.6545894, 1.6543981,
       1.6629694, 1.6577289, 1.6532335, 1.653304 , 1.6575725, 1.6572138,
       1.6532729, 1.6534877, 1.6549022, 1.6616033, 1.6534863, 1.6532737,
       1.6560062, 1.6558799, 1.6535478, 1.6534234, 1.6555641, 1.6575181,
       1.6594315, 1.6603496, 1.6537803, 1.6536366, 1.6565648, 1.6543278,
       1.658775 , 1.6533257, 1.6549313, 1.6534997, 1.6537828, 1.6561908,
       1.6532314, 1.6535609, 1.6532377, 1.6533946, 1.654083 , 1.6535087,
       1.6541888, 1.6535038, 1.6546324, 1.6595786, 1.6532863, 1.6535718,
       1.6588881, 1.6536165, 1.6549766, 1.6532346, 1.6541944, 1.6532596,
       1.655606 , 1.6564696, 1.6535218, 1.6542243, 1.6548387, 1.6539

### Backward propagation

In [85]:
optimizer = tf.train.AdamOptimizer(learning_rate = alpha)
tf.set_random_seed(0)
model.load_weights('Initialized_model.h5')
for i in range(1):
    with tf.GradientTape() as tape:
        z_2, z_mean, z_logsigma = model(train_inputs)
        losses = total_vae_loss(train_inputs, z_2, z_mean, z_logsigma)
        grads = tape.gradient(losses['total_vae_loss'], model.weights) # model.weights means variables here.
        optimizer.apply_gradients(zip(grads, model.weights))

In [86]:
# See if we can agree on rc-loss first.
kl_weight = 5e-3
tf.set_random_seed(0)
total_loss, cache = forward_propagation_K(train_inputs, learned_params)
sampled_vector = cache['sampled_vector']
z_2 = cache['z_2']
W_2 = cache['W_2']
eps = cache['eps']
a_1 = cache['a_1']
z_1 = cache['z_1']
W_mu = cache['W_mu']
z_mu = cache['z_mu']
W_sigma = cache['W_sigma']
inputs = cache['inputs']

grad_z_2_rc = 2* (z_2 - train_inputs) # (n_x, m)
grad_W_2_rc = (1/m) * sampled_vector.T.dot(grad_z_2_rc) # (n_z, n_x)
grad_b_2_rc = np.mean(grad_z_2_rc, axis=0, keepdims = True)
grad_s_rc =  grad_z_2_rc.dot(W_2.T) # (n_z, m), gradient of sampled vector
grad_z_mu_rc = grad_s_rc # (n_z, m)
grad_z_sigma_rc = np.multiply(grad_s_rc, 0.5 * np.exp(z_sigma * 0.5) * eps)
grad_W_mu_rc =  (1/m) * a_1.T.dot(grad_z_mu_rc)
grad_b_mu_rc = np.mean(grad_z_mu_rc, axis = 0,keepdims=True)
grad_W_sigma_rc = (1/m) * a_1.T.dot(grad_z_sigma_rc)
grad_b_sigma_rc = np.mean(grad_z_sigma_rc, axis = 0, keepdims=True)

grad_a_1_rc = grad_z_mu_rc.dot(W_mu.T) + grad_z_sigma_rc.dot(W_sigma.T)
grad_z_1_rc = np.multiply(grad_a_1_rc, relu(z_1, derivative=True))
grad_W_1_rc = (1/m) * inputs.T.dot(grad_z_1_rc)
grad_b_1_rc = np.mean(grad_z_1_rc, axis = 0, keepdims = True)

# This is the kl-loss gradients
grad_b_2_kl = 0
grad_W_2_kl = 0
grad_z_sigma_kl =  0.5*(np.exp(z_sigma) - 1)
grad_z_mu_kl = z_mu
grad_W_sigma_kl = a_1.T.dot(grad_z_sigma_kl)
grad_b_sigma_kl = np.sum(grad_z_sigma_kl, axis = 0)
grad_W_mu_kl = a_1.T.dot(grad_z_mu_kl)
grad_b_mu_kl = np.sum(grad_z_mu_kl, axis = 0)
grad_a_1_kl = grad_z_mu_kl.dot(W_mu.T) + grad_z_sigma_kl.dot(W_sigma.T)
grad_z_1_kl = np.multiply(grad_a_1_kl, relu(z_1, derivative = True))
grad_W_1_kl = inputs.T.dot(grad_z_1_kl)
grad_b_1_kl = np.sum(grad_z_1_kl, axis = 0, keepdims = True)

# Add up the two gradients in the correct mix:
grad_W_2 = m * grad_W_2_rc + kl_weight * grad_W_2_kl
grad_b_2 = m * grad_b_2_rc + kl_weight * grad_b_2_kl
grad_W_mu = m * grad_W_mu_rc + kl_weight * grad_W_mu_kl
grad_b_mu = m * grad_b_mu_rc + kl_weight * grad_b_mu_kl
grad_W_sigma = m * grad_W_sigma_rc + kl_weight * grad_W_sigma_kl
grad_b_sigma = m * grad_b_sigma_rc + kl_weight * grad_b_sigma_kl
grad_W_1 = m * grad_W_1_rc + kl_weight * grad_W_1_kl
grad_b_1 = m * grad_b_1_rc + kl_weight * grad_b_1_kl

In [48]:
grads[5],grads[4]

(<tf.Tensor: id=1496, shape=(10,), dtype=float32, numpy=
 array([15.215241  ,  9.842631  , 34.036102  ,  0.04143097, 27.469017  ,
        30.452288  ,  1.4018004 , 16.87278   ,  3.5270784 , -0.19807675],
       dtype=float32)>,
 <tf.Tensor: id=1498, shape=(20, 10), dtype=float32, numpy=
 array([[ 8.35354447e-01, -7.23051846e-01,  1.02119684e+00,
         -4.08361852e-01,  2.74458075e+00,  3.31166816e+00,
         -1.02228999e-01,  7.57703543e-01,  8.21092948e-02,
         -3.08054034e-02],
        [ 1.17832577e+00, -1.01991498e+00,  1.44046962e+00,
         -5.76022804e-01,  3.87142229e+00,  4.67133713e+00,
         -1.44201055e-01,  1.06879377e+00,  1.15821004e-01,
         -4.34531830e-02],
        [ 3.14388007e-01,  4.58721250e-01,  7.50095427e-01,
         -4.30460423e-02,  4.13787872e-01,  3.65335196e-01,
          3.47389653e-02,  3.78335118e-01,  6.30216226e-02,
          8.80144886e-04],
        [ 5.44171333e-01,  7.93996513e-01,  1.29833317e+00,
         -7.45080560e-02,  7.16

In [49]:
# grad_b_mu*1000 + grad_b_mu_kl * 5e-3 , \
# grad_W_mu*1000 + grad_W_mu_kl * 5e-3
grad_b_sigma, grad_W_sigma

(array([[15.2152403 ,  9.84263355, 34.03610569,  0.04143069, 27.46901667,
         30.45229175,  1.4018    , 16.87278543,  3.52707815, -0.1980767 ]]),
 array([[ 8.35354562e-01, -7.23051893e-01,  1.02119706e+00,
         -4.08361892e-01,  2.74458094e+00,  3.31166809e+00,
         -1.02229021e-01,  7.57703625e-01,  8.21093711e-02,
         -3.08054048e-02],
        [ 1.17832566e+00, -1.01991494e+00,  1.44046942e+00,
         -5.76022826e-01,  3.87142213e+00,  4.67133797e+00,
         -1.44201138e-01,  1.06879362e+00,  1.15820974e-01,
         -4.34531642e-02],
        [ 3.14387982e-01,  4.58721231e-01,  7.50095313e-01,
         -4.30460557e-02,  4.13787846e-01,  3.65335146e-01,
          3.47389503e-02,  3.78335121e-01,  6.30216206e-02,
          8.80144578e-04],
        [ 5.44171335e-01,  7.93996460e-01,  1.29833324e+00,
         -7.45080314e-02,  7.16221667e-01,  6.32355324e-01,
          6.01293372e-02,  6.54856864e-01,  1.09083557e-01,
          1.52343435e-03],
        [ 3.34605910e

### Adam optimizer

In [62]:
model.get_weights()[5] # b_2

array([-0.10000001, -0.10000002, -0.10000001, -0.09999925, -0.1       ,
       -0.10000001, -0.09999997, -0.10000001, -0.09999999,  0.09999985],
      dtype=float32)

In [63]:
moment = np.divide(first_moment_update(0, grad_b_sigma, 0.9, 1),
                  (np.sqrt(second_moment_update(0, grad_b_sigma, 0.999, 1)) + 1e-8))
modified_alpha = alpha * (np.sqrt(1-np.power(0.999,1))/(1-np.power(0.9,1)))
modified_alpha*moment

array([[ 0.03162278,  0.03162278,  0.03162278,  0.03162277,  0.03162278,
         0.03162278,  0.03162278,  0.03162278,  0.03162278, -0.03162278]])

In [61]:
grads[5], grad_b_sigma # Both are 354.732

(<tf.Tensor: id=1496, shape=(10,), dtype=float32, numpy=
 array([15.215241  ,  9.842631  , 34.036102  ,  0.04143097, 27.469017  ,
        30.452288  ,  1.4018004 , 16.87278   ,  3.5270784 , -0.19807675],
       dtype=float32)>,
 array([[15.2152403 ,  9.84263355, 34.03610569,  0.04143069, 27.46901667,
         30.45229175,  1.4018    , 16.87278543,  3.52707815, -0.1980767 ]]))

In [83]:
def first_moment_update(previous_moment, grad, beta, timestep):
    biased = beta * previous_moment + (1 - beta) * grad
#     unbiased = biased / (1 - np.power(beta, timestep))
    return biased

def second_moment_update(previous_moment, grad, beta, timestep):
    biased = beta * previous_moment + (1 - beta) * np.square(grad)
#     unbiased = biased / (1 - np.power(beta,timestep))
    return biased

first_moment_update(0, grad_b_2, 0.9,1)/10, second_moment_update(0, grad_b_2, 0.999,1)/1000

(array([[-0.06388587]]), array([[4.08140457e-05]]))

In [77]:
optimizer._get_beta_accumulators()

(<tf.Variable 'beta1_power:0' shape=() dtype=float32, numpy=0.7289999>,
 <tf.Variable 'beta2_power:0' shape=() dtype=float32, numpy=0.9970031>)

In [87]:
optimizer.get_slot(model.trainable_variables[7],'m'), \
optimizer.get_slot(model.trainable_variables[7],'v')

(<tf.Variable 'dense_3/bias/Adam_14:0' shape=(1,) dtype=float32, numpy=array([-0.6388595], dtype=float32)>,
 <tf.Variable 'dense_3/bias/Adam_15:0' shape=(1,) dtype=float32, numpy=array([0.0408136], dtype=float32)>)

In [88]:
grads[7]

<tf.Tensor: id=3356, shape=(1,), dtype=float32, numpy=array([-6.3885937], dtype=float32)>

In [90]:
first_moment_update(0, grads[7], 0.9,2), second_moment_update(0, grads[7], 0.999,2)

(<tf.Tensor: id=3605, shape=(1,), dtype=float32, numpy=array([-0.6388594], dtype=float32)>,
 array([0.04081413], dtype=float32))